# Imports

In [2]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft, rfftfreq
from sklearn.metrics import accuracy_score
import librosa
import resampy
import librosa.display
import IPython.display as ipd
import os
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pydub import AudioSegment
import os
warnings.filterwarnings('ignore')


# Model

### define features extraction function

In [3]:
def mfcc_feature_extractor(audio,sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T,axis=0)
    
    return mfccsScaledFeatures

In [4]:
def contrast_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate) 
    contrastScaled = np.mean(contrast.T,axis=0)
    
    return contrastScaled

In [5]:
def tonnetz_feature_extractor(audio,sampleRate):
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T,axis=0)
    
    return tonnetzScaled

In [6]:
def centroid_feature_extractor(audio,sampleRate):
    centroid = librosa.feature.spectral_centroid(y=audio,sr=sampleRate) 
    centroidScaled = np.mean(centroid.T,axis=0)
    
    return centroidScaled

In [7]:
def chroma_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft,sr=sampleRate)
    chromaScaled = np.mean(chroma.T,axis=0)
    
    return chromaScaled

In [8]:
def features_extractor(file):
    if file.lower().endswith(".m4a"):
        audio = AudioSegment.from_file(file, format="m4a")
        file_name, _ = os.path.splitext(os.path.basename(file))
        output_path = os.path.join(os.path.dirname(file), file_name + ".wav")
        audio.export(output_path, format="wav")
        os.remove(file)
        file = output_path

    features = []
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfcc = mfcc_feature_extractor(audio, sample_rate)
    contrast = contrast_feature_extractor(audio, sample_rate)
    tonnetz = tonnetz_feature_extractor(audio, sample_rate)
    chroma = chroma_feature_extractor(audio, sample_rate)

    # Concatenate individual feature arrays
    concatenated_features = np.concatenate((mfcc, contrast, tonnetz, chroma), axis=0)
    features.append(concatenated_features)

    return features

### define add features function

In [9]:
def add_features(extractedFeatures, dirPath, label):
    for file in os.listdir(dirPath):
        filePath = os.path.join(dirPath, file)

        if filePath.lower().endswith(".m4a"):
            audio = AudioSegment.from_file(filePath, format="m4a")
            file_name, _ = os.path.splitext(os.path.basename(filePath))
            output_path = os.path.join(os.path.dirname(filePath), file_name + ".wav")
            audio.export(output_path, format="wav")
            os.remove(filePath)
            filePath = output_path

        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc = mfcc_feature_extractor(audio, sampleRate)
        contrast = contrast_feature_extractor(audio, sampleRate)
        tonnetz = tonnetz_feature_extractor(audio, sampleRate)
        chroma = chroma_feature_extractor(audio, sampleRate)

        extractedFeatures.append([mfcc, contrast, tonnetz, chroma, label])

### creating dictionary with dirPath and label

In [10]:
dict1 = {
    "ibrahim": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ibrahim",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ibrahim",
    ],
    "ahmeda": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmeda",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmeda",
    ],
    "omar": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/omar",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar",
    ],
    "hazem": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hazem",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hazem",
    ],
    "ahmedk": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/ahmedk",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/ahmedk",
    ],
    "hassan": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/hassan",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/hassan",
    ],
    "mohannad": [
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/unlock/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/grant/mohannad",
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/mohannad",
    ],
    "other":[
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people", 
        "C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer", 
    ]
}


In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pydub import AudioSegment
import librosa

features = []
target = []

for person, paths in dict1.items():
    person_features = []
    person_target = []

    for path in paths:
        audio_files = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.wav')]

        for audio_file in audio_files:
            audio, sampleRate = librosa.load(audio_file, res_type='kaiser_fast') 
            mfcc = mfcc_feature_extractor(audio, sampleRate)
            contrast = contrast_feature_extractor(audio, sampleRate)
            tonnetz = tonnetz_feature_extractor(audio, sampleRate)
            chroma = chroma_feature_extractor(audio, sampleRate)

            combined_features = np.concatenate((mfcc, contrast, tonnetz, chroma))
            person_features.append(combined_features)
            person_target.append(person)

    person_features = np.array(person_features)
    person_target = np.array(person_target)

    encoder = LabelEncoder()
    person_target_encoded = encoder.fit_transform(person_target)

    xTrain_person, xTest_person, yTrain_person, yTest_person = train_test_split(person_features, person_target_encoded, test_size=0.25, random_state=0)

    features.append((xTrain_person, xTest_person))
    target.append((yTrain_person, yTest_person))

In [23]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

model = SVC(kernel='linear',decision_function_shape="ovo" ,probability=True) 

# Initialize lists to store results
train_accuracies = []
test_accuracies = []

xTrain = []
yTrain = []
xTest = []
yTest = []

cnt = 0
for (xTrain_person, xTest_person), (yTrain_person, yTest_person) in zip(features, target):
    # encoder.fit(np.concatenate((yTrain_person, yTest_person)))

    # Encode the target values
    yTrain_person_encoded = [cnt]*len(yTrain_person)
    yTest_person_encoded = [cnt]*len(yTest_person)

    # Append the encoded values to the lists
    xTrain.extend(xTrain_person)

    yTrain.extend(yTrain_person_encoded)
    xTest.extend(xTest_person)
    yTest.extend(yTest_person_encoded)
    cnt += 1

xTrain = np.array(xTrain)
yTrain = np.array(yTrain)
xTest = np.array(xTest)
yTest = np.array(yTest)

model.fit(xTrain, yTrain)

SVC(decision_function_shape='ovo', kernel='linear', probability=True)

In [13]:
preds = model.predict(xTest)
print(preds)

[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3
 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 6 6 6 6 6 6
 1 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 3 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


### making classification report to see the result of the predicton 

In [24]:
print(classification_report(yTest, preds))

train_predictions = model.predict(xTrain)
test_predictions = model.predict(xTest)

train_accuracy = accuracy_score(yTrain, train_predictions)
test_accuracy = accuracy_score(yTest, test_predictions)

print(train_accuracy, "  ", test_accuracy)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.90      1.00      0.95         9
           2       1.00      1.00      1.00        11
           3       0.93      1.00      0.96        13
           4       1.00      1.00      1.00        15
           5       1.00      1.00      1.00         8
           6       1.00      0.89      0.94         9
           7       1.00      0.99      1.00       123

    accuracy                           0.99       200
   macro avg       0.98      0.99      0.98       200
weighted avg       0.99      0.99      0.99       200

1.0    0.99


In [28]:
def prediction(pred):   
    # {'ahmeda': 0, 'ahmedk': 1, 'hassan': 2, 'hazem': 3, 'ibrahim': 4, 'mohannad': 5, 'omar': 6, 'other': 7}
    if pred == 0:
        return ("Ibrahim")
    elif pred == 1:
        return ("Ahmed Ali")
    elif pred == 2:
        return ("Omar")
    elif pred == 3:
        return ("Hazem")
    elif pred == 4:
        return ("Ahmed Khaled")
    elif pred == 5:
        return ("Hassan")
    elif pred == 6:
        return ("Mohannad")
    elif pred == 7:
        return ("other")

In [39]:
def my_predict(features):
    probabilities = model.predict_proba(features)
    # print(probabilities)

    for i in range(len(features)):
        class_pred_index = probabilities[i].argmax()  
        class_pred = probabilities[i, class_pred_index]  
        threshold = 0.50
        
        if class_pred > threshold:
            return class_pred_index
        else:
            return 7

In [49]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad"):
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/open/mohannad/{file}") 
  print(model.predict_proba(testFeatures))
  print("1: ",prediction(model.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[0.04399021 0.0217501  0.00816423 0.01978406 0.01181497 0.04700039
  0.82757854 0.0199175 ]]
1:  Mohannad
2:  Mohannad
[[0.01857277 0.00615221 0.00464626 0.01482326 0.00513683 0.02166985
  0.92105864 0.0079402 ]]
1:  Mohannad
2:  Mohannad
[[0.01327129 0.02243843 0.00669895 0.0395322  0.00377535 0.10256148
  0.79727411 0.01444819]]
1:  Mohannad
2:  Mohannad
[[0.02100185 0.00905084 0.00511214 0.02711047 0.00598528 0.0388388
  0.86837896 0.02452166]]
1:  Mohannad
2:  Mohannad
[[0.02634542 0.0082216  0.0074197  0.03746523 0.01100207 0.01537474
  0.86924672 0.02492452]]
1:  Mohannad
2:  Mohannad
[[0.02965031 0.01256703 0.00472119 0.0187002  0.00870373 0.01613081
  0.89892394 0.01060279]]
1:  Mohannad
2:  Mohannad
[[0.02138672 0.00910395 0.00437224 0.01305108 0.00556966 0.02868697
  0.90409074 0.01373864]]
1:  Mohannad
2:  Mohannad
[[0.01478319 0.00834848 0.00464938 0.01036252 0.0052822  0.03886866
  0.89822076 0.01948482]]
1:  Mohannad
2:  Mohannad
[[0.02230331 0.01195391 0.01164516 0.0125

In [48]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other sentences/omar/{file}") 
  print(model.predict_proba(testFeatures))
  print("1: ",prediction(model.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[1.17976382e-03 1.23355795e-02 9.62094401e-01 2.56280249e-03
  1.28370996e-03 9.29826765e-04 1.83824301e-02 1.23148603e-03]]
1:  Omar
2:  Omar
[[0.01264359 0.01550909 0.83586002 0.07985801 0.00712375 0.00942688
  0.02515555 0.01442311]]
1:  Omar
2:  Omar
[[0.01392018 0.02348888 0.80137922 0.04201763 0.00456054 0.01375412
  0.09057193 0.01030751]]
1:  Omar
2:  Omar
[[0.00381413 0.03698727 0.85947792 0.0453894  0.00166214 0.02597103
  0.01644119 0.01025692]]
1:  Omar
2:  Omar
[[0.00497831 0.02083141 0.90354081 0.00565141 0.00128891 0.0129678
  0.04930406 0.0014373 ]]
1:  Omar
2:  Omar


In [47]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/other people/{file}") 
  print(model.predict_proba(testFeatures))
  print("1: ",prediction(model.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[3.18794823e-04 5.48402404e-05 1.84003086e-03 6.27247546e-03
  3.48438252e-03 2.75000906e-05 8.80047088e-05 9.87913971e-01]]
1:  other
2:  other
[[7.08243477e-04 3.39698235e-04 8.20524424e-03 1.85157710e-03
  3.35502366e-03 1.21752803e-04 4.89672412e-04 9.84928788e-01]]
1:  other
2:  other
[[3.95231738e-04 1.86708419e-04 1.22015904e-02 2.67735509e-03
  4.64434141e-03 8.57246388e-05 3.08062142e-04 9.79500986e-01]]
1:  other
2:  other
[[3.33022401e-04 8.85896308e-05 4.16225118e-03 1.70247469e-03
  1.83704223e-03 3.35741945e-05 1.54896987e-04 9.91688149e-01]]
1:  other
2:  other
[[3.40712080e-05 2.40768861e-05 1.36490003e-03 4.45918896e-03
  3.13268600e-03 2.54653442e-05 9.34906678e-05 9.90866121e-01]]
1:  other
2:  other


In [50]:
testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/dataset/recorded/1.wav") 
print(model.predict_proba(testFeatures))
print("1: ",prediction(model.predict(testFeatures)))
print("2: ",prediction(my_predict(testFeatures))) 

[[0.00542062 0.02160502 0.58568306 0.03300999 0.00553833 0.01401241
  0.04448048 0.29025008]]
1:  Omar
2:  Omar


In [46]:
for file in os.listdir("C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer")[0:5]:
  testFeatures = features_extractor(f"C:/Users/omara/OneDrive/Desktop/Voice Signal Authentication/ML/new audios/tamer/{file}") 
  print(model.predict_proba(testFeatures))
  print("1: ",prediction(model.predict(testFeatures)))
  print("2: ",prediction(my_predict(testFeatures))) 

[[2.43261359e-06 4.39763214e-06 6.74147573e-06 1.39979894e-05
  2.08059623e-03 8.08442466e-08 1.40970655e-05 9.97877656e-01]]
1:  other
2:  other
[[1.14787949e-05 2.66286797e-05 1.39599563e-04 9.38420050e-04
  3.56430414e-03 1.10915948e-06 2.58189015e-05 9.95292641e-01]]
1:  other
2:  other
[[7.51392469e-04 1.07774333e-04 7.05120787e-04 4.62849912e-03
  1.25609186e-02 2.54553551e-05 1.03690382e-04 9.81117149e-01]]
1:  other
2:  other
[[4.30079450e-05 4.03772902e-05 4.55172650e-04 2.64181185e-04
  9.04026745e-03 6.32186464e-07 5.20480804e-05 9.90104313e-01]]
1:  other
2:  other
[[2.32283107e-06 4.81327065e-05 1.24006509e-04 3.15925598e-05
  1.05204185e-03 2.76630636e-07 1.96567167e-05 9.98721970e-01]]
1:  other
2:  other


In [45]:
pickle_out = open("processing.pkl", "wb")
pickle.dump(model, pickle_out)
pickle_out.close()